# Live Demonstration

In order to confirm that the model was trained sufficiently, a live demonstration is needed.

### Model
Instantiate a neural network for the collision avoidance inference.

In [ ]:
import torch
import torchvision
from torch2trt import TRTModule

In [ ]:
# Load the model from file.
device = torch.device('cuda')
model_trt = TRTModule()
model_trt.load_state_dict(torch.load('collision_avoidance.pth'))

### View

Display the camera feed with controls for its speed an a status of if the path forward is ``free`` or ``blocked``.

In [ ]:
import torchvision.transforms as transforms
import torch.nn.functional as F
import cv2
import PIL.Image
import numpy as np

# Prepare the image data for input to the model.

mean = torch.Tensor([0.485, 0.456, 0.406]).cuda().half()
std = torch.Tensor([0.229, 0.224, 0.225]).cuda().half()

normalize = torchvision.transforms.Normalize(mean, std)

def preprocess(image):
    image = PIL.Image.fromarray(image)
    image = transforms.functional.to_tensor(image).to(device).half()
    image.sub_(mean[:, None, None]).div_(std[:, None, None])
    return image[None, ...]

Great! We've now defined our pre-processing function which can convert images from the camera format to the neural network input format.

Now, let's start and display our camera.  You should be pretty familiar with this by now.  We'll also create a slider that will display the
probability that the robot is blocked.  We'll also display a slider that allows us to control the robot's base speed.

In [ ]:
import traitlets
from IPython.display import display
import ipywidgets.widgets as widgets
from jetbot import Camera, bgr8_to_jpeg

# Display the camera, widgets to control the robot and feedback.

camera = Camera.instance(width=224, height=224)
image = widgets.Image(format='jpeg', width=224, height=224)
blocked_slider = widgets.FloatSlider(description='blocked', min=0.0, max=1.0, orientation='vertical')
speed_slider = widgets.FloatSlider(description='speed', min=0.0, max=0.5, value=0.0, step=0.01, orientation='horizontal')

camera_link = traitlets.dlink((camera, 'value'), (image, 'value'), transform=bgr8_to_jpeg)

display(widgets.VBox([widgets.HBox([image, blocked_slider]), speed_slider]))

### Robot

Control the robot per  ``free`` or ``blocked`` detection and user command of speed.

In [ ]:
from jetbot import Robot

robot = Robot()

In [ ]:
import torch.nn.functional as F
import time

def update(change):
    # Update the speed of the wheels.
    global blocked_slider, robot
    x = change['new'] 
    x = preprocess(x)
    y = model_trt(x)
    
    y = F.softmax(y, dim=1)
    
    prob_blocked = float(y.flatten()[0])
    
    blocked_slider.value = prob_blocked
    
    if prob_blocked < 0.5:
        # Move forward 
        robot.forward(speed_slider.value)
    else:
        # Redirect the robot if it is blocked
        robot.left(speed_slider.value)
    
    time.sleep(0.001)
        
# Update wheels per camera stream.
update({'new': camera.value})
camera.observe(update, names='value') 

In [ ]:
import time

# Stop the robot if it can't communicate with the network.
camera.unobserve(update, names='value')
time.sleep(0.1)
robot.stop()

### Cleanup
Stop the camera before closing notebook so that others can access it.

In [ ]:
camera.stop()